# Estimating default and asset correlation with method of moments and maximum likelihood method

In this Python notebook, we reproduce the numerical results of chapter 6 "Modelling and Estimating Default Correlation with the Asset Value Approach" in Credit Risk Modeling using Excel and VBA, by Gunter Loffler and Peter Posch 

https://www.wiley.com/en-gb/Credit+Risk+Modeling+using+Excel+and+VBA%2C+2nd+Edition-p-9780470660928.

Remarkably, the authors carried out the original work in excel! It is so much easier to implement the same algos in python. However, as the algos are numerically involved, we are not surprised to find our implementation also slow. 

There are two main assuptions behind the original work: 1) All obligors share the same defaut and joint default probabilities as the other obligors in the same credit group, either Investment Grade or Specuative Grade. 2) There is no serial correlation in the time series of defaults, so defaults in any year are not affected by the defaults in a previous year.

For all the mathematical derivations and formulae, please look at chapter 6. 

In [1]:
import pandas as pd
import numpy  as np

from scipy.stats     import norm
from scipy.optimize  import root
from scipy.integrate import quad
from scipy.optimize  import minimize
from scipy.special   import comb
from scipy.stats     import norm

import matplotlib.pyplot as plt
%matplotlib inline

Let's read our data set, from the book:

In [2]:
DefaultData = pd.read_csv ('DefaultHistory.csv', index_col=False)
print (DefaultData.head(2))
print (DefaultData.tail(2))
size = len(DefaultData)

   Year  IGDefaults  SpeculativeGradeDefaults  IG_No  SpeculativeGrade_No
0  1981           0                         2   1064                  321
1  1982           2                        15   1093                  340
    Year  IGDefaults  SpeculativeGradeDefaults  IG_No  SpeculativeGrade_No
27  2008          14                        88   3398                 2528
28  2009          11                       223   3445                 2415


In [3]:
print (DefaultData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
Year                        29 non-null int64
IGDefaults                  29 non-null int64
SpeculativeGradeDefaults    29 non-null int64
IG_No                       29 non-null int64
SpeculativeGrade_No         29 non-null int64
dtypes: int64(5)
memory usage: 1.2 KB
None


### Methods of Moments

#### Default Rate

We estimate the average default rate for the Investment Grades and Speculative credit categories, $ p_{ig}$ and $p_{spec}$ by averaging their respective annual default rates

In [4]:
DefaultData['DefaultRateIG']     = DefaultData.IGDefaults              /DefaultData.IG_No
DefaultData['DefaultRateSpec']   = DefaultData.SpeculativeGradeDefaults/DefaultData.SpeculativeGrade_No

In [5]:
AverageDefaultRateIG   = np.mean(DefaultData['DefaultRateIG'])
AverageDefaultRateSpec = np.mean(DefaultData['DefaultRateSpec'])

In [6]:
print(AverageDefaultRateIG)
print(AverageDefaultRateSpec)

0.0011440217358502029
0.04369411291593368


#### Joint default rate

We now estimate the average joint default rates as we previously did for the default rates. 

We relate the number of observed joint (pair) defaults to the total number of possible joint (pair) defaults as follows.

Let's indicate with $D_t$ the number of defaults in year $t$ and with $N_t$ the number of exposures at the beginning of year $t$

The number of observed joint (pair) defaults is
$$ {{D_t}\choose 2} =  \frac{D_t(D_t -1)}{2} $$
$$ $$
The total number of possible joint (pair) defaults is
$$ {{N_t}\choose 2} =  \frac{N_t(N_t -1)}{2} $$
$$ $$
The joint default rate for year $t$ is
$$ p_{joint, t}=  \frac{D_t(D_t -1)}{N_t(N_t -1)} $$
$$ $$
and the average joint default rate for all $t's$ is 
$$p_{joint} = \frac{1}{T}\sum_{t=1}^T p_{joint, t}=\frac{1}{T}\sum_{t=1}^T \frac{D_t(D_t -1)}{N_t(N_t -1)}$$

In [7]:
DefaultData['Joint_Def_ProbIG']     = DefaultData.IGDefaults * (DefaultData.IGDefaults -1) \
/(DefaultData.IG_No*(DefaultData.IG_No -1))

DefaultData['Joint_Def_ProbSpec']   = DefaultData.SpeculativeGradeDefaults *(DefaultData.SpeculativeGradeDefaults - 1) \
/(DefaultData.SpeculativeGrade_No*(DefaultData.SpeculativeGrade_No-1))

In [8]:
JointDefProbIG   = np.mean(DefaultData['Joint_Def_ProbIG'])
JointDefProbSpec = np.mean(DefaultData['Joint_Def_ProbSpec'])

print(np.round(JointDefProbIG,6))
print(np.round(JointDefProbSpec,6))

2e-06
0.002624


#### Default correlation 

The default correlation is specified by the individual and joint default probabilities. So using the average default and joint default probabilities, we calculate the default correlation as

$$\rho_{i,j}= \frac{p_{i,j}-p_ip_j}{\sqrt{p_i(1-p_j)p_j(1-p_i)}} $$

Again, as stated earlier, we assume all obligors in the same category, share the same default and joint default probability 

In [9]:
DefaultCorrIG   = (JointDefProbIG - AverageDefaultRateIG*AverageDefaultRateIG)\
/(np.sqrt(AverageDefaultRateIG*(1-AverageDefaultRateIG)*AverageDefaultRateIG*(1-AverageDefaultRateIG)))

DefaultCorrSpec = (JointDefProbSpec - AverageDefaultRateSpec*AverageDefaultRateSpec)\
/(np.sqrt(AverageDefaultRateSpec*(1-AverageDefaultRateSpec)*AverageDefaultRateSpec*(1-AverageDefaultRateSpec)))

#### The asset value model

Instead of imposing the default correlation structure "directly" from the historic default data, more conveniently defaults can represented as function of continuous "random" variables and then a joint default structure is imposed on these variables. 

These variables are often referred as latent variables and are interpreted as the firm's asset value. When the asset value $A_i$ drops below a critical value $d_i$ (called default threshod), default is triggered.

The default indicator for the obligor $i$ can be represented as
$$Default_i = 1 \, ,if \, A_i <= d_i$$

whereas the no default indicator is 

$$No \, default_i = 0 \, ,if \, A_i > d_i$$

The joint default probability is instead written as 
$$Prob(A_i <= d_i \, , A_j <= d_j)$$

Continuing, the asset value $A_i$ depends on the common factor $Z$, which is common to all obligors, on the idiosyncratic factor $\epsilon_i$ and on the factor sensitivity $w_i$

$$A_{i} = w_iZ+\sqrt{1-w_i^2}\epsilon_i$$

The asset correlation between $i$ and $j$ is completely determined by their factor sensitivities $w_i$ and $w_j$ and simplifies to

$$\rho_{i,j}^{asset}= w_iw_j$$

The default probability is given by
$$P[A_i<=d_i] = p_i = \Phi(d_i)$$

and the joint default probability is given by
$$P[A_i<=d_i,A_j<=d_j ] = p_{i,j} = \Phi_{2}(d_i, d_j,\rho_{i,j}^{asset})$$

where 
$\Phi(d_i)$ is the cumulative standard normal distribution function and
$$ $$
$\Phi_{2}(d_i, d_j,\rho_{i,j}^{asset})$ is the bivariate standard normal distribution function with asset correlation $\rho_{i,j}^{asset}$ 


Once a credit is allocated to one of the two credit categories (Investment Grades or Speculative), it will share the same default rate, joint defaut rate, critical value (or default threshold), default and asset correlation of all the other credits in the same category. It is a big assumption indeed!

With those assumptions in mind it is easy to find the asset correlation, $ \rho_{i,j}^{asset}$, that allows us to match the joint default rate as calculated earlier.

We introduce the following three functions to parametrize the asset model

In [10]:
def tetrachoric(x , y , rho): 

    FACCURACY = 0.0000000000001
    MinStopK  = 20    
    hx  = 1
    hy  = 1
    hx1 = 0
    hy1 = 0
    k   = 0
    c   = rho
    z   = c
    s   = z
    CheckPass = 0

    while (CheckPass < MinStopK):
        k   = k + 1
        hx2 = hx1
        hy2 = hy1
        hx1 = hx
        hy1 = hy
        hx  = x * hx1 - (k - 1) * hx2
        hy  = y * hy1 - (k - 1) * hy2
        c   = c * rho / (k + 1)
        z   = hx * hy * c
        s   = s + z
        if (abs(z / s) < FACCURACY):
            CheckPass = CheckPass + 1
        else:
            CheckPass = 0
        
    return s

def bivnor(x , y , rho):
    biv = 0.0
    if (rho == 0):
        biv = norm.cdf(x) * norm.cdf(y)
    else:
        biv = norm.cdf(x) * norm.cdf(y) + norm.pdf(x) * norm.pdf(y) * tetrachoric(x, y, rho)
    return biv

def wrapperRoot(x, y, jointDefault):
    def f(rho):
        return bivnor(x,y,rho) - jointDefault
    return f

#### Default Triggers

are below

In [11]:
x_inverseDRIG   = norm.ppf(AverageDefaultRateIG,   loc=0, scale=1)
x_inverseDRSpec = norm.ppf(AverageDefaultRateSpec, loc=0, scale=1)
print(x_inverseDRIG)
print(x_inverseDRSpec)

-3.0500485880438437
-1.7093387152616


In [12]:
# Let's find now the asset correlation which matches the joint default rate for the IG credit group
print("Joint Default rate for the IG credit is", JointDefProbIG)
ff  = wrapperRoot(x_inverseDRIG,x_inverseDRIG,JointDefProbIG)
sol = root(ff,0.05)
AssetCorrIG = sol.x[0]
print("Asset Correlation  for the IG credit is", AssetCorrIG)

Joint Default rate for the IG credit is 2.1991576201508045e-06
Asset Correlation  for the IG credit is 0.04883917450262377


In [13]:
# so assuming an asset correlation of 0.0488, the joint default probability is the same as above
print(bivnor(x_inverseDRIG,x_inverseDRIG,0.04883917450262377))
print(JointDefProbIG)

2.1991576201508045e-06
2.1991576201508045e-06


In [14]:
# Let's do the same for the speculative credit category
ff  = wrapperRoot(x_inverseDRSpec,x_inverseDRSpec,JointDefProbSpec)
sol = root(ff,0.05)
AssetCorrSpec = sol.x[0]

Let' print the moments so far calcuated, which we remind were estimated directly on the historic default data

In [15]:
print("{0:16} {1:18} {2:24} {3:8}".format("Credit Type", "Default Prob", "Joint Default Prob", "Default Correlation"))

print('{0} {1:22.4f} {2:21.8f} {3:28.5}'.format ("IG",AverageDefaultRateIG, JointDefProbIG, DefaultCorrIG))
print('{0} {1:13.4f} {2:21.8f} {3:26.5}'.format ("Speculative",AverageDefaultRateSpec, JointDefProbSpec, DefaultCorrSpec))

Credit Type      Default Prob       Joint Default Prob       Default Correlation
IG                 0.0011            0.00000220                   0.00077917
Speculative        0.0437            0.00262395                   0.017106


Whereas when using an asset model to describe a default, the moments are below

In [16]:
print("{0:16} {1:20} {2:22} {3:8}".format("Credit Type", "Default Threshold", "Joint Default Prob", "Asset Correlation"))

print('{0} {1:22.4f} {2:26.8f} {3:21.5}'.format ("IG",x_inverseDRIG, JointDefProbIG, AssetCorrIG))
print('{0} {1:13.4f} {2:26.8f} {3:21.5}'.format ("Speculative",x_inverseDRSpec, JointDefProbSpec, AssetCorrSpec))

Credit Type      Default Threshold    Joint Default Prob     Asset Correlation
IG                -3.0500                 0.00000220              0.048839
Speculative       -1.7093                 0.00262395              0.074955


### Estimating Asset Correlation with Maximum Likelihood - One factor

Alternatively, we can apply the maximum likelihood method to parametrize an asset model. In a nutshell, with the MLM we determine the default probability and the factor sensitivity, such that the probability of observing the historical default rate is maximised.  

Formally, we write the default probability of obligor $i$ associated to a given realization of the factor $Z$ as

$$p_i(Z) = P\left[A_i<=\Phi^{-1}(p_i)|Z\right]$$
$$ $$
$$p_i(Z) =\Phi{\left(\frac{\Phi^{-1}(p_i) - w_iZ}{\sqrt{1-w^2}}\right)}$$

Conditional on the realization of the factor $Z$, defaults are independent.
If the conditional default probability is the same across all obligors at $p(Z)$, and defaults at time $t$ are independent from defaults at time $t-1$, then the total number of defaults $D$ follows a binomial distribution with success probabiity $p(Z)$ and $N$ trials.

Applying the binomial formula $${{N}\choose{k}} \cdot p^kq^{N-k}$$

leads to the following log likelihood function for the number of defaults for the group $k$ = $\{$Investment and Speculative Grade$\}$, in year $t=$\{1,..,$T$}

$$ln(L_k) = \sum_{t=1}^T ln\int_{-\infty}^\infty {{N_{kt}}\choose{D_{kt}}} \cdot p_k(Z)^{D_{kt}}(1-p_k(Z))^{N_{kt}-D_{kt}}d\Phi(Z)$$

The log likelihood function will be applied to the two credit groups indipendently as follows. 

Let's write the binomial expression, $p(Z)$ and log likelhood function in python

In [17]:
def Pz(p,w,z):
    num = norm.ppf(p)-np.multiply(w,z)
    pG  = norm.cdf(np.divide(num,np.sqrt(1-w*w)))
    return pG

def CMFz(z, Rho, p, N, K):
    pg  = Pz(p,Rho, z)    
    f   = comb(N,K)*np.power(pg,K)*np.power(1-pg,N-K)
    cmf = f        
    return cmf

def logL(Rho_p, T, nVec, kVec):
    Rho  = Rho_p[0]
    p    = Rho_p[1]    
    L    = np.zeros(T)

    for t in range(0,T):
        ss = np.array([CMFz(s, Rho, p, nVec[t], kVec[t]) for s in x])
        for i in range(0,size):            
            L[t] =  np.dot(ss,y)
    logL = -np.sum(np.log(L))
    return logL

In [18]:
h = 0.1
x = [-5 + i * h for i in range(101)] 
y = np.array([norm.pdf(s)*h for s in x])

Let's check that the array $y$ is indeed a probability density. It should sum to one

In [19]:
np.round(np.sum(y),7) # close enough

0.9999996

Let's see if we can replicate the log likelihood function value as it was reported in the book.

In the book the probability and factor sensitivity for the sector IG were below. 

In [20]:
pp    = 0.00121497751624143
wi    = 0.276457395273792
Rho_p = (wi,pp)

In [21]:
# for t = 1981, it agrees!
print(logL(Rho_p, 1, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))

0.8487256141381895


In [22]:
# for for the full IG data set it agrees too!
print(logL(Rho_p, 29, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))

60.27009497803652


Let's see if we can find the same probability and factor sensitivity for the sector IG by maximizing the log likelihood function.

In [23]:
X1       = np.array(DefaultData.IG_No)
X2       = np.array(DefaultData.IGDefaults)
theta    = np.ones(2)
theta[0] = 0.20
theta[1] = AverageDefaultRateIG

In [24]:
res_IG = minimize(logL, theta, args=(29,X1,X2), method='TNC',jac=None, options={'maxiter':50}) 
print (res_IG)
# they are very close!

     fun: 60.27009491039413
     jac: array([0.00558771, 0.07108838])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 26
     nit: 8
  status: 1
 success: True
       x: array([0.27644787, 0.00121494])


#### Problems with the "scipy.special.comb" for the Speculative credit group

The function comb returns $\infty$ when the number $k$ is too large. The problem was already highlighted by the authors who also provided an alternative algo, called sterling approximation 

In [25]:
import math
from math import pi, log, exp

def lncombin(n, k):
    res = 0
    if(comb(n,k) == math.inf):
        if (k == 0) :
            res = 0
        else:
            lfn = (n + 0.5) * log(n) - n + 0.5 * log(2 * pi)
            lfk = (k + 0.5) * log(k) - k + 0.5 * log(2 * pi)
            lfnk = ((n - k) + 0.5) * log(n - k) - (n - k) + 0.5 * log(2 * pi)
            res = lfn - lfk - lfnk    

    else:
        res = log(comb(n,k))
    return res

# test it!
lncombin(2415,223)

740.0467215389326

In [26]:
# Let's rewrite the previous function where we use the sterling approximation.
def CMFzSterling(z, Rho, p, N, K):
    pg  = Pz(p,Rho, z)    
    f   = lncombin(N,K) + log(pg)*K + log(1-pg)*(N-K)
    cmf = exp(f)        
    return cmf

def logLSterling(Rho_p, T, nVec, kVec):
    Rho  = Rho_p[0]
    p    = Rho_p[1]    
    L    = np.zeros(T)
    for t in range(0,T):
        ss = np.array([CMFzSterling(s, Rho, p, nVec[t], kVec[t]) for s in x])
        for i in range(0,size):            
            L[t] =  np.dot(ss,y)
    logL = -np.sum(np.log(L))
    return logL

Let's now run the MLM on the Speculative credit group

In [27]:
X1       = np.array(DefaultData.SpeculativeGrade_No)
X2       = np.array(DefaultData.SpeculativeGradeDefaults)
theta    = np.ones(2)
theta[0] = 0.15
theta[1] = AverageDefaultRateSpec
res_Spec = minimize(logLSterling, theta, args=(29,X1,X2), method='TNC',jac=None, options={'maxiter':50}) 
print (res_Spec)

     fun: 132.24879725873734
     jac: array([-0.00841567,  0.02413856])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 45
     nit: 9
  status: 1
 success: True
       x: array([0.29139055, 0.04385974])


### Estimating Asset Correlation with Maximum Likelihood - Two factors

Alternatively, we could run the MLM on all data we have available, IG and Speculative, in one go, and estimate the default probabilities and factor sensitivities for the two sectors. 

The joint probability of observing IG and Speculative grade defaulting is estimated for a given realization on $Z$, which is common to IG and Speculative credits. So we can still use the binomial formula, as once we know $Z$, we will be dealing with independent events.

In [28]:
def CMFzSterling2Factors(z, Rho1, p1, Rho2, p2, N1, K1, N2, K2):
    pg1  = Pz(p1,Rho1, z)
    pg2  = Pz(p2,Rho2, z)
    
    f1   = lncombin(N1,K1) + log(pg1)*K1 + log(1-pg1)*(N1-K1)
    f2   = lncombin(N2,K2) + log(pg2)*K2 + log(1-pg2)*(N2-K2)
    cmf = exp(f1 + f2)        
    return cmf

def logLSterling2Factors(Rho_p, T, nVec1, kVec1, nVec2, kVec2):
    Rho1  = Rho_p[0]
    p1    = Rho_p[1]    
    Rho2  = Rho_p[2]
    p2    = Rho_p[3]    
    L    = np.zeros(T)
    
    for t in range(0,T):
        ss = np.array([CMFzSterling2Factors(s, Rho1, p1, Rho2, p2, nVec1[t], kVec1[t], nVec2[t], kVec2[t]) for s in x])
        for i in range(0,size):            
            L[t] =  np.dot(ss,y)
    logL = -np.sum(np.log(L))
    return logL

In [29]:
X1_IG = np.array(DefaultData.IG_No)
X2_IG = np.array(DefaultData.IGDefaults)

X1_Sp = np.array(DefaultData.SpeculativeGrade_No)
X2_Sp = np.array(DefaultData.SpeculativeGradeDefaults)

theta  = np.ones(4)
theta[0]= 0.20
theta[1]= AverageDefaultRateIG
theta[2]= 0.15
theta[3] = AverageDefaultRateSpec

res_2Fact = minimize(logLSterling2Factors, theta, args=(29,X1_IG,X2_IG, X1_Sp,X2_Sp), \
                     method='TNC',jac=None, options={'maxiter':50}) 
print (res_2Fact)

     fun: 186.5334705044338
     jac: array([-15.73734778,  69.67679553,  -0.11623342,  24.73203438])
 message: 'Max. number of function evaluations reached'
    nfev: 50
     nit: 10
  status: 3
 success: False
       x: array([0.20353734, 0.0012931 , 0.27907251, 0.04347081])


Let's report the MLM values so far calculated

In [30]:
print("{0:13} {1:21} {2:23} {3:22}".format(" ", "Method of Moments", "Maximum Likelihood 1F", "Maximum Likelihood 2F's"))

print('{0} {1:21.4f} {2:26.5f} {3:21.5f}'.format("IG p",    AverageDefaultRateIG,  res_IG.x[1],      res_2Fact.x[1]))
print('{0} {1:19.4f} {2:26.5f} {3:21.5f}'.format("IG rho",  AssetCorrIG,           res_IG.x[0]**2,   res_2Fact.x[0]**2))
print('{0} {1:19.4f} {2:26.5f} {3:21.5f}'.format("Spec p",  AverageDefaultRateSpec,res_Spec.x[1],    res_2Fact.x[3]))
print('{0} {1:17.4f} {2:26.5f} {3:21.5f}'.format("Spec rho",AssetCorrSpec,         res_Spec.x[0]**2, res_2Fact.x[2]**2))

              Method of Moments     Maximum Likelihood 1F   Maximum Likelihood 2F's
IG p                0.0011                    0.00121               0.00129
IG rho              0.0488                    0.07642               0.04143
Spec p              0.0437                    0.04386               0.04347
Spec rho            0.0750                    0.08491               0.07788


We find a benefit when using a MLM with 2 factors vs MLM with 1 factor: MLM-2F's $p'$s and rho's are much closer to the respective values calculated with Methods of Moments    

For completeness, we compare our runs against those reported in the book.

In [31]:
print("{0:14} {1:25} {2:23} {3:22} {4:22}".format(" ", "Book MLM 1F","Book MLM 2F", "MLM 1F", "MLM 2F"))

print('{0} {1:21.5f} {2:25.5}  {3:22.5} {4:22.5}'.format ("IG p",0.00121498,0.001374488, res_IG.x[1],res_2Fact.x[1]))
print('{0} {1:19.5f} {2:26.8f} {3:21.5} {4:22.5}'.format ("IG rho",  0.076428691, 0.071192177, res_IG.x[0]**2,res_2Fact.x[0]**2))
print('{0:19} {1:6}  {2:23.6f} {3:22.5f}{4:23.5f}'.format ("Spec p",  "NA", 0.044464189, res_Spec.x[1], res_2Fact.x[3]))
print('{0:19} {1:6}  {2:25.8f} {3:21.5} {4:22.5}'.format ("Spec rho","NA", 0.107009136, res_Spec.x[0]**2,res_2Fact.x[2]**2))
print('{0:17} {1:6.4f} {2:23.4f} {3:23.4f} {4:20.5}'.format ("MLM Value-IG",60.2700527, 185.7244, res_IG.fun, res_2Fact.fun))

               Book MLM 1F               Book MLM 2F             MLM 1F                 MLM 2F                
IG p               0.00121                 0.0013745               0.0012149              0.0012931
IG rho             0.07643                 0.07119218              0.076423               0.041427
Spec p              NA                     0.044464                0.04386                0.04347
Spec rho            NA                     0.10700914              0.084908               0.077881
MLM Value-IG      60.2701                185.7244                 60.2701               186.53


### Integrate with scipy.integrate import quad

Running the MLM to find model coefficients takes a very long time. The algo is numerically intense. 

Do we save calculation time if we replace, in in the function logL(), the numerical solution of the integral

$$ln(L_k) = \sum_{t=1}^T ln\int_{-\infty}^\infty {{N_{kt}}\choose{D_{kt}}} \cdot p_k(Z)^{D_{kt}}(1-p_k(Z))^{N_{kt}-D_{kt}}d\Phi(z)$$

with the quad object from scipy.integrate ?

First we modify CMFz and logL as follows:

In [32]:
def CMFzquad(z, Rho, p, N, K):
    pg  = Pz(p,Rho, z)    
    f   = comb(N,K)*np.power(pg,K)*np.power(1-pg,N-K)
    cmf = f*norm.pdf(z)        
    return cmf

def logLquad(Rho_p, T, nVec, kVec):
    Rho  = Rho_p[0]
    p    = Rho_p[1]    
    L    = np.zeros(T)    
    
    for t in range(0,T):
        fCMF = lambda s, Rho, p, nX, kX: CMFzquad(s, Rho,p,nX,kX)
        L[t],err = quad(fCMF,-5,5, args=(Rho, p, nVec[t], kVec[t]))
    logL = -np.sum(np.log(L))
    return logL          

Let's test the algo on the IG data for year, $t$ = 1981, and see if it it agrees with our previous results!

In [33]:
# and it does
print(logLquad(Rho_p, 1, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))
print(logL(Rho_p, 1, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))

0.8487257730543333
0.8487256141381895


and for the full IG data set

In [34]:
# It agrees too!
print(logLquad(Rho_p, 29, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))
print(logL(Rho_p, 29, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))

60.2700972395946
60.27009497803652


And let's find the ML parameters for the IG dataset 

In [35]:
X1       = np.array(DefaultData.IG_No)
X2       = np.array(DefaultData.IGDefaults)
theta    = np.ones(2)
theta[0] = 0.20
theta[1] = AverageDefaultRateIG

res_IG_quad = minimize(logLquad, theta, args=(29,X1,X2), method='TNC',jac=None, options={'maxiter':50}) 
print(res_IG_quad)

     fun: 60.27009711920736
     jac: array([0.00163638, 0.00667342])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 27
     nit: 8
  status: 1
 success: True
       x: array([0.27643282, 0.00121492])


and see how it compares with our previous results 

In [36]:
# Parameter vaues are very close!!
print (res_IG_quad.x)
print (res_IG.x)

[0.27643282 0.00121492]
[0.27644787 0.00121494]


In [37]:
# and so is the MLM value!!
print (res_IG_quad.fun)
print (res_IG.fun)

60.27009711920736
60.27009491039413


#### Is there any time saving?

In [38]:
import numpy as np
import timeit

In [41]:
%timeit (logLquad(Rho_p, 29, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))

1.24 s ± 32.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%timeit (logL(Rho_p, 29, np.array(DefaultData.IG_No), np.array(DefaultData.IGDefaults)))

765 ms ± 57.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The above shows that surprisingly, scipy.integrate.quad is slower!

In the following two lines, we also tested the calculation time on the minimize algo with and without quad. The testing is very slow and it does take sometime to complete, so do not run it unless you really need it. 

In [39]:
%timeit minimize(logLquad, theta, args=(29,X1,X2), method='TNC',jac=None, options={'maxiter':20}) 

1min 40s ± 3.83 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%timeit minimize(logL, theta, args=(29,X1,X2), method='TNC',jac=None, options={'maxiter':20}) 

The slowest run took 89.43 times longer than the fastest. This could mean that an intermediate result is being cached.
13min 27s ± 30min 30s per loop (mean ± std. dev. of 7 runs, 1 loop each)
